In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# video_batch, classes = next(iter(train_dataloader))

In [1]:
import torch
import sys
sys.path.append('/content/drive/MyDrive/thesis/src')
from data import create_dataloaders
import torchvision
import torch.nn as nn 
import torch.optim as optim
from engine import train

# DEVICE:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# HYPERPARAMETERS:
EPOCHS_NUM = 5
BATCH_SIZE = 32
LEARNING_RATE = 0.001

# PRETRAINED MODEL:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
backbone_transform = weights.transforms()
backbone = torchvision.models.efficientnet_b0(weights=weights).to(DEVICE)
output_shape = 50# TODO find a way to get it automatically from whatever dataset is the input.
backbone.classifier = torch.nn.Sequential(torch.nn.Dropout(p=0.2, inplace=True), torch.nn.Linear(in_features=1280, out_features=output_shape, bias=True)
).to(DEVICE)

for param in backbone.features.parameters():
  param.requires_grad = False

torch.manual_seed(38)
torch.cuda.manual_seed(38)

# PATHS:
TRAINING_DIRECTORY = "/content/drive/MyDrive/thesis/data/HMDB-51-downsampled_copy"
TEST_DIRECTORY = "/content/drive/MyDrive/thesis/data/HMDB-51-downsampled_copy"

# DATALOADERS:
training_dataloader, test_dataloader, class_names = create_dataloaders(training_directory=TRAINING_DIRECTORY, test_directory=TEST_DIRECTORY, transform=backbone_transform, batch_size=BATCH_SIZE)

#LOSS & OPTIMISER 
loss_function = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(backbone.parameters(), lr=LEARNING_RATE)

#TRAINING...
train(model=backbone, training_dataloader=training_dataloader, test_dataloader=test_dataloader, optimiser=optimiser, loss_function=loss_function, epochs=EPOCHS_NUM, device=DEVICE, pretrained=True)


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 56.6MB/s]


  0%|          | 0/5 [00:00<?, ?it/s]

IndexError: ignored